In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (SparkSession.builder
         .appName('halltape_pyspark_local')
         .getOrCreate()
)

print(f'Активные Spark сессии: {spark.sparkContext.uiWebUrl}')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/21 14:28:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Активные Spark сессии: http://fedora:4040


25/05/21 14:28:31 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
PATH = 'data/customs_data.csv'

In [3]:
spark.read.csv(PATH).show()

+--------------------+
|                 _c0|
+--------------------+
|month;country;cod...|
|01/2016;IT;620469...|
|01/2016;CN;900190...|
|01/2016;BY;841430...|
|01/2016;US;901850...|
|01/2016;EE;902110...|
|01/2016;FR;381600...|
|01/2016;MX;852351...|
|01/2016;JP;620452...|
|01/2016;KR;611020...|
|01/2016;KG;852713...|
|01/2016;ZA;842123...|
|01/2016;CN;851810...|
|01/2016;TR;841790...|
|01/2016;IT;390610...|
|01/2016;CZ;870840...|
|01/2016;ES;640419...|
|01/2016;IT;940490...|
|01/2016;UA;820780...|
|01/2016;CN;330410...|
+--------------------+
only showing top 20 rows



In [4]:
spark.read.csv(PATH, sep=';', header=True).show(5)

+-------+-------+----------+------+-----+--------+------+--------+-------------+-----------+--------------------+
|  month|country|      code| value|netto|quantity|region|district|direction_eng|measure_eng|           load_date|
+-------+-------+----------+------+-----+--------+------+--------+-------------+-----------+--------------------+
|01/2016|     IT|6204695000|   131|    1|       7| 46000|      01|           IM|        ShT|2024-07-01T00:00:...|
|01/2016|     CN|9001900009|112750|   18|       0| 46000|      01|           IM|          1|2024-01-01T00:00:...|
|01/2016|     BY|8414302004|   392|   57|       8| 50000|      06|           IM|        ShT|2024-06-01T00:00:...|
|01/2016|     US|9018509000| 54349|  179|       0| 40000|      02|           IM|          1|2024-04-01T00:00:...|
|01/2016|     EE|9021101000| 17304|  372|       0| 46000|      01|           IM|          1|2024-02-01T00:00:...|
+-------+-------+----------+------+-----+--------+------+--------+-------------+--------

In [7]:
df = spark.read.csv(PATH, sep=';', header=True, inferSchema=True)

df.show(truncate=False)

+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+-------------------+
|month  |country|code      |value |netto |quantity|region|district|direction_eng|measure_eng|load_date          |
+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+-------------------+
|01/2016|IT     |6204695000|131   |1     |7       |46000 |1       |IM           |ShT        |2024-07-01 00:00:00|
|01/2016|CN     |9001900009|112750|18    |0       |46000 |1       |IM           |1          |2024-01-01 00:00:00|
|01/2016|BY     |8414302004|392   |57    |8       |50000 |6       |IM           |ShT        |2024-06-01 00:00:00|
|01/2016|US     |9018509000|54349 |179   |0       |40000 |2       |IM           |1          |2024-04-01 00:00:00|
|01/2016|EE     |9021101000|17304 |372   |0       |46000 |1       |IM           |1          |2024-02-01 00:00:00|
|01/2016|FR     |3816000000|323488|253600|0       |40000 |2       |IM           |1      

In [8]:
df.printSchema()

root
 |-- month: string (nullable = true)
 |-- country: string (nullable = true)
 |-- code: string (nullable = true)
 |-- value: string (nullable = true)
 |-- netto: long (nullable = true)
 |-- quantity: long (nullable = true)
 |-- region: integer (nullable = true)
 |-- district: integer (nullable = true)
 |-- direction_eng: string (nullable = true)
 |-- measure_eng: string (nullable = true)
 |-- load_date: timestamp (nullable = true)



In [9]:
result = (df.withColumnRenamed("direction_eng", "direction")
            .withColumnRenamed("measure_eng", "measure")
         )

result.printSchema()

root
 |-- month: string (nullable = true)
 |-- country: string (nullable = true)
 |-- code: string (nullable = true)
 |-- value: string (nullable = true)
 |-- netto: long (nullable = true)
 |-- quantity: long (nullable = true)
 |-- region: integer (nullable = true)
 |-- district: integer (nullable = true)
 |-- direction: string (nullable = true)
 |-- measure: string (nullable = true)
 |-- load_date: timestamp (nullable = true)



In [10]:
result.columns

['month',
 'country',
 'code',
 'value',
 'netto',
 'quantity',
 'region',
 'district',
 'direction',
 'measure',
 'load_date']

In [11]:
result.select('country').distinct().show(10, truncate=False)

+-------+
|country|
+-------+
|LT     |
|MM     |
|DZ     |
|CI     |
|TC     |
|FI     |
|SC     |
|AZ     |
|UA     |
|RO     |
+-------+
only showing top 10 rows



In [17]:
(result.groupBy('country').agg(F.count('*')
       .alias('total_rows')).orderBy(F.col('total_rows').desc()).show()
)

+-------+----------+
|country|total_rows|
+-------+----------+
|     BY|   3509568|
|     KZ|   2519896|
|     CN|   2454792|
|     DE|   1542311|
|     UA|   1158498|
|     IT|   1102837|
|     US|    835936|
|     PL|    666690|
|     FR|    593040|
|     JP|    571756|
|     TR|    463432|
|     KR|    446907|
|     GB|    443091|
|     AM|    438705|
|     CZ|    407360|
|     KG|    403565|
|     ES|    401644|
|     IN|    374151|
|     NL|    365193|
|     UZ|    329707|
+-------+----------+
only showing top 20 rows



In [19]:
df_de = (result.where(F.col('country') == 'DE')
               .where(F.col('value').isNotNull())
)

In [20]:
df_de.show()

+-------+-------+----------+------+-----+--------+------+--------+---------+-------+-------------------+
|  month|country|      code| value|netto|quantity|region|district|direction|measure|          load_date|
+-------+-------+----------+------+-----+--------+------+--------+---------+-------+-------------------+
|01/2016|     DE|4016995709|  5901|  172|       0| 46000|       1|       IM|      1|2024-01-01 00:00:00|
|01/2016|     DE|8708809109|  1213|   94|       0| 45000|       1|       IM|      1|2024-01-01 00:00:00|
|01/2016|     DE|7013419000|  7020| 1611|     492| 45000|       1|       IM|    ShT|2024-02-01 00:00:00|
|01/2016|     DE|3923309090| 46294| 8048|       0| 45000|       1|       IM|      1|2024-04-01 00:00:00|
|01/2016|     DE|4015190000|440008|73357| 6463937| 45000|       1|       IM|    PAR|2024-03-01 00:00:00|
|01/2016|     DE|2918140000|   265|   15|       0| 45000|       1|       IM|      1|2024-03-01 00:00:00|
|01/2016|     DE|3208101000| 10075|  680|       0| 7800

In [24]:
final = df_de.select('month',
                    'country',
                    'code',
                    'value',
                    'netto',
                    'quantity',
                    'region',
                    'district',
                    'direction',
                    'measure',
                    F.col('load_date').cast('date')
                    )
final.printSchema()

root
 |-- month: string (nullable = true)
 |-- country: string (nullable = true)
 |-- code: string (nullable = true)
 |-- value: string (nullable = true)
 |-- netto: long (nullable = true)
 |-- quantity: long (nullable = true)
 |-- region: integer (nullable = true)
 |-- district: integer (nullable = true)
 |-- direction: string (nullable = true)
 |-- measure: string (nullable = true)
 |-- load_date: date (nullable = true)



In [25]:
# Сохранение неконтроллируемое по кол-ву файлов
final\
    .write\
    .format('csv')\
    .options(header='True', sep=';')\
    .csv('data/final_no_control')

partition_num = final.rdd.getNumPartitions()
print(f'Кол-во партиций {partition_num}')

Кол-во партиций 16


In [26]:
# Сохранение контроллируемое по кол-ву файлов - ОДИН ФАЙЛ
final\
    .coalesce(1)\
    .write\
    .format('csv')\
    .options(header='True', sep=';')\
    .csv('data/final_one_file') 

partition_num = final.coalesce(1).rdd.getNumPartitions()
print(f'Кол-во партиций {partition_num}')

Кол-во партиций 1


In [27]:

# Сохранения с партицированием
final\
    .write\
    .partitionBy('load_date')\
    .format('csv')\
    .options(header='True', sep=';')\
    .csv('data/final_partitioned')

print_df = final.select('load_date').distinct()
print(f'Load_date distinct: {print_df.count()}')

Load_date distinct: 10


In [28]:
# Сохранения с партицированием и repartition внутри самой партиции
final\
    .repartition(1, 'load_date')\
    .write\
    .partitionBy('load_date')\
    .format('csv')\
    .options(header='True', sep=';')\
    .csv('data/final_partitioned_repart')

partition_num = final.repartition(1, 'load_date').rdd.getNumPartitions()
print(f'Кол-во партиций {partition_num}')

Кол-во партиций 1


## Read Transformed

In [29]:
read_no_control = (spark.read.csv('data/final_no_control/',
                                  header=True,
                                  sep=';').where(F.col('load_date') == '2024-01-01'))

In [30]:
read_no_control.count()

350998

In [31]:
read_final_one_file = (spark.read.csv('data/final_one_file/',
                                  header=True,
                                  sep=';').where(F.col('load_date') == '2024-01-01'))

read_final_one_file.count()


350998

In [32]:
read_partitioned = (spark.read.csv('data/final_partitioned',
                                  header=True,
                                  sep=';').where(F.col('load_date') == '2024-01-01'))

read_partitioned.count()


350998

In [33]:
read_partitioned_repart = (spark.read.csv('data/final_partitioned_repart',
                                  header=True,
                                  sep=';').where(F.col('load_date') == '2024-01-01'))

read_partitioned_repart.count()


350998

## JOIN

In [34]:
data = [
    (14000, "Северный"),
    (11000, "Южный"),
    (10000, "Восточный"),
    (26000, "Западный"),
    (56000, "Центральный")
]

region_df = spark.createDataFrame(data, schema='region_id long, name string')

region_df.show()

customs_data = spark.read.csv('data/customs_data.csv', header=True, sep=';')

customs_data.show(10)

+---------+-----------+
|region_id|       name|
+---------+-----------+
|    14000|   Северный|
|    11000|      Южный|
|    10000|  Восточный|
|    26000|   Западный|
|    56000|Центральный|
+---------+-----------+

+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+--------------------+
|  month|country|      code| value| netto|quantity|region|district|direction_eng|measure_eng|           load_date|
+-------+-------+----------+------+------+--------+------+--------+-------------+-----------+--------------------+
|01/2016|     IT|6204695000|   131|     1|       7| 46000|      01|           IM|        ShT|2024-07-01T00:00:...|
|01/2016|     CN|9001900009|112750|    18|       0| 46000|      01|           IM|          1|2024-01-01T00:00:...|
|01/2016|     BY|8414302004|   392|    57|       8| 50000|      06|           IM|        ShT|2024-06-01T00:00:...|
|01/2016|     US|9018509000| 54349|   179|       0| 40000|      02|           IM|          1|

In [35]:
import time
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [37]:
start_time = time.time()

customs_data.join(region_df, customs_data.region == region_df.region_id, "left").count()

end_time = time.time()

print(f'Elapsed time for join operation: {end_time - start_time:.2F} seconds')

Elapsed time for join operation: 116.59 seconds


In [38]:
start_time = time.time()

customs_data.join(F.broadcast(region_df), customs_data.region == region_df.region_id, "left").count()

end_time = time.time()

print(f"Elapsed time for broadcast join operation: {end_time - start_time:.2f} seconds")

Elapsed time for broadcast join operation: 60.71 seconds


## Cache | Persist

In [39]:
customs_data.cache().count()


25/05/21 16:46:06 WARN MemoryStore: Not enough space to cache rdd_248_7 in memory! (computed 48.6 MiB so far)
25/05/21 16:46:07 WARN BlockManager: Persisting block rdd_248_7 to disk instead.
25/05/21 16:46:09 WARN MemoryStore: Not enough space to cache rdd_248_7 in memory! (computed 48.6 MiB so far)
25/05/21 16:46:25 WARN MemoryStore: Not enough space to cache rdd_248_9 in memory! (computed 19.7 MiB so far)
25/05/21 16:46:25 WARN BlockManager: Persisting block rdd_248_9 to disk instead.
25/05/21 16:46:39 WARN MemoryStore: Not enough space to cache rdd_248_10 in memory! (computed 9.9 MiB so far)
25/05/21 16:46:39 WARN BlockManager: Persisting block rdd_248_10 to disk instead.
25/05/21 16:46:42 WARN MemoryStore: Not enough space to cache rdd_248_9 in memory! (computed 10.2 MiB so far)
25/05/21 16:46:46 WARN MemoryStore: Not enough space to cache rdd_248_11 in memory! (computed 9.8 MiB so far)
25/05/21 16:46:46 WARN BlockManager: Persisting block rdd_248_11 to disk instead.
25/05/21 16:46

26392290

In [40]:
customs_data.unpersist()

DataFrame[month: string, country: string, code: string, value: string, netto: string, quantity: string, region: string, district: string, direction_eng: string, measure_eng: string, load_date: string]

## Repartition & Coalesce

In [41]:
data = [(1,'one'), (2,'two'), (3,'three'), (4,'four'),
        (5,'five'), (6,'six'), (7, 'seven'), (8, 'eight'),
        (9, 'nine')]

df = spark.createDataFrame(data, ['id', 'number'])

df.show()

+---+------+
| id|number|
+---+------+
|  1|   one|
|  2|   two|
|  3| three|
|  4|  four|
|  5|  five|
|  6|   six|
|  7| seven|
|  8| eight|
|  9|  nine|
+---+------+



In [44]:
mix = df.repartition(6)
mix.rdd.glom().collect()

[[Row(id=2, number='two'), Row(id=7, number='seven')],
 [Row(id=6, number='six')],
 [Row(id=5, number='five')],
 [Row(id=1, number='one')],
 [Row(id=4, number='four'), Row(id=8, number='eight')],
 [Row(id=3, number='three'), Row(id=9, number='nine')]]

In [45]:
mix.coalesce(3).rdd.glom().collect()

[[Row(id=2, number='two'), Row(id=7, number='seven'), Row(id=6, number='six')],
 [Row(id=3, number='three'),
  Row(id=9, number='nine'),
  Row(id=1, number='one')],
 [Row(id=5, number='five'),
  Row(id=4, number='four'),
  Row(id=8, number='eight')]]

In [47]:
mix.toPandas().head()

ModuleNotFoundError: No module named 'distutils'

In [48]:
spark.stop()